In [3]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0

x_train_flat = x_train.reshape(-1, 3072)
x_test_flat = x_test.reshape(-1, 3072)

class RBM(tf.keras.Model):
    def __init__(self, n_visible, n_hidden):
        super().__init__()
        self.W = tf.Variable(tf.random.normal([n_visible, n_hidden], stddev=0.01))
        self.hb = tf.Variable(tf.zeros([n_hidden]))
        self.vb = tf.Variable(tf.zeros([n_visible]))

    def sample(self, probs):
        return tf.nn.relu(tf.sign(probs - tf.random.uniform(tf.shape(probs))))

    def forward(self, v):
        h_prob = tf.nn.sigmoid(tf.matmul(v, self.W) + self.hb)
        h_sample = self.sample(h_prob)
        return h_sample

    def train_step(self, v, lr=0.01):
        h_prob = tf.nn.sigmoid(tf.matmul(v, self.W) + self.hb)
        h_sample = self.sample(h_prob)
        v_recon_prob = tf.nn.sigmoid(tf.matmul(h_sample, tf.transpose(self.W)) + self.vb)
        h_recon_prob = tf.nn.sigmoid(tf.matmul(v_recon_prob, self.W) + self.hb)

        pos_grad = tf.matmul(tf.transpose(v), h_prob)
        neg_grad = tf.matmul(tf.transpose(v_recon_prob), h_recon_prob)

        self.W.assign_add(lr * (pos_grad - neg_grad) / tf.cast(tf.shape(v)[0], tf.float32))
        self.vb.assign_add(lr * tf.reduce_mean(v - v_recon_prob, axis=0))
        self.hb.assign_add(lr * tf.reduce_mean(h_prob - h_recon_prob, axis=0))

def train_rbm_layerwise(x_train, layer_sizes):
    outputs = x_train
    rbms = []
    for i, size in enumerate(layer_sizes):
        rbm = RBM(outputs.shape[1], size)
        print(f"\nTraining RBM layer {i+1} with {size} hidden units...")
        for epoch in range(5):
            print(f"Epoch {epoch+1} for RBM layer {i+1}...")
            for i in range(0, outputs.shape[0], 64):
                batch = outputs[i:i+64]
                rbm.train_step(batch)
        rbms.append(rbm)
        outputs = rbm.forward(outputs)
    return rbms, outputs

layer_sizes = [1024, 512]

print("Training the first RBM (1024 hidden units)...")
rbms, final_features = train_rbm_layerwise(x_train_flat, layer_sizes)

print("\nFinished training all RBM layers. Now fine-tuning with supervised learning...\n")

x_train_transformed = final_features
x_test_transformed = rbms[1].forward(rbms[0].forward(x_test_flat))

model = tf.keras.Sequential([
    tf.keras.Input(shape=(512,)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print("Training the final model using fine-tuning on transformed features...\n")
model.fit(x_train_transformed, y_train, epochs=10, batch_size=64, validation_split=0.1, verbose=0)

print("\nEvaluating the model on the test data...\n")
preds = np.argmax(model.predict(x_test_transformed), axis=1)
acc = accuracy_score(y_test, preds)

print(f"DBN Accuracy on CIFAR-10: {acc:.4f}")


Training the first RBM (1024 hidden units)...

Training RBM layer 1 with 1024 hidden units...
Epoch 1 for RBM layer 1...
Epoch 2 for RBM layer 49985...
Epoch 3 for RBM layer 49985...
Epoch 4 for RBM layer 49985...
Epoch 5 for RBM layer 49985...

Training RBM layer 2 with 512 hidden units...
Epoch 1 for RBM layer 2...
Epoch 2 for RBM layer 49985...
Epoch 3 for RBM layer 49985...
Epoch 4 for RBM layer 49985...
Epoch 5 for RBM layer 49985...

Finished training all RBM layers. Now fine-tuning with supervised learning...

Training the final model using fine-tuning on transformed features...


Evaluating the model on the test data...

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
DBN Accuracy on CIFAR-10: 0.3712
